# MVP: Engenharia de Dados
### PUC - Rio | Ciência de Dados e Analytics
Nome: Romário Silva de Almeida

Matrícula: 4052025000761

Dataset: Google Play Store Apps

### Descrição do Problema
O dataset contém dados de aplicativos da Play Store. O sucesso de um aplicativo depende de muitas variáveis. A análise destes dados têm um enorme potencial para impulsionar o sucesso de empresas de desenvolvimento de aplicativos, através de insights valiosos para que os desenvolvedores possam direcionar seus trabalhos e buscarem o sucesso no mercado.

### Perguntas a serem respondidas

Para fins de atingir o objetivo apresentado, as perguntas estão direcionadas para uma análise por categoria. A categorização torna mais fácil para os usuários encontrarem os aplicativos que precisam. Outras perguntas em relação ao preço para instalação e aos aplicativos também serão investigadas: 

1. Quais categorias possuem maiores quantidades de aplicativos disponíveis no catálogo da Google Play?
2. Alguma categoria se destaca em termos de classificação?
3. E em relação ao número de avaliações, quais possuem maior engajamento dos usuários?
4. Qual a distribuição de downloads por categoria?
5. Alguns aplicativos são pagos e outros gratuitos. Quais são os mais baixados em cada tipo?
6. E o engajamento, qual tipo possui maior quantidade de avaliações em relação à quantidade downloads?

---

### Como será feito?

**Metodologia baseada no Modelo _Medallion_**

**_Camada Bronze_:** 
Responsável pela ingestão dos dados brutos, que são armazenados no formato Delta Lake na tabela _bronze.app_base_, preservando a estrutura original. Esta camada atua como fonte primária, garantindo uma cópia fiel dos dados de origem.

**_Camada Silver_:**
Etapa de processamento e refinamento dos dados brutos. Inclui:

Limpeza: tratamento de valores nulos e correção de inconsistências.
Validação e transformação: conversão de tipos, padronização de atributos.

**_Camada Gold_:**
Focada na agregação e sumarização dos dados para análises avançadas. 
As tabelas são criadas no schema gold. Essas estruturas contêm resultados prontos para consumo por ferramentas de BI, relatórios ou dashboards, respondendo diretamente às principais perguntas de negócio.

-   **Ferramentas:**
    -   **Databricks:** Plataforma unificada para desenvolvimento, execução e gerenciamento do pipeline de dados.
    -   **Apache Spark (SQL e PySpark):** Engine de processamento distribuído utilizado para manipulação eficiente de grandes volumes de dados e execução das transformações e análises.
    -   **Delta Lake:** Formato de armazenamento otimizado para as tabelas em todas as camadas, oferecendo ACID, versionamento e performance.
    

## 1. Setup Inicial: Limpeza e Criação de Schemas
Limpa execuções anteriores (se existirem) e cria os schemas para as camadas Bronze, Silver e Gold.

In [0]:

%sql
DROP CATALOG IF EXISTS mvp CASCADE

In [0]:

%sql
CREATE CATALOG mvp

In [0]:

%sql
USE CATALOG mvp

In [0]:

%sql
DROP DATABASE IF EXISTS bronze CASCADE;
DROP DATABASE IF EXISTS silver CASCADE;
DROP DATABASE IF EXISTS gold CASCADE;

In [0]:

%sql
CREATE SCHEMA IF NOT EXISTS bronze COMMENT 'Camada para dados brutos ingeridos';
CREATE SCHEMA IF NOT EXISTS silver COMMENT 'Camada para dados limpos, filtrados e enriquecidos';
CREATE SCHEMA IF NOT EXISTS gold COMMENT 'Camada para dados agregados e prontos para análise';

In [0]:
from pyspark.sql import functions as F, Window
from pyspark.sql.functions import sha2, concat_ws, col, count

%md
## 2. Camada Bronze: Ingestão de Dados Brutos
**Objetivo:** Armazenar os dados brutos do dataset Google Play App Store em seu formato original, servindo como base para as transformações nas camadas seguintes.

**Onde os dados foram obtidos?**

Utilizei o site Kaggle.com para realizar a busca deste dataset.
O link para a página no Kaggle é: https://www.kaggle.com/lava18/google-play-store-apps

**License**
CC BY-SA 4.0: Livre para 
Compartilhar — copie e redistribua o material em qualquer meio ou formato para qualquer finalidade, inclusive comercial.
Adaptar — remixar, transformar e desenvolver o material para qualquer finalidade, inclusive comercial.

**Como foi feito o processo de ingestão dos dados?**

Devido a restrições de segurança do computador utilizado, não foi possível fazer o download diretamente no site do Kaggle.
Por estar mais habituado, o arquivo .csv foi salvo em um repositório no Github, ao invés de utilizar o upload manual diretamente no Databricks.
O acesso e download foi realizado através de uma função nativa do Databricks, armazenando na camada DBFS. Com o dataset dentro do mesmo ambiente, foi criado um Dataframe Spark, normalizado os nomes de colunas (continham espaços em branco entre palavras) e persistido como uma tabela Delta (_bronze.app_base_).

In [0]:

%sql
USE CATALOG mvp;
USE SCHEMA bronze

In [0]:
%sql
CREATE VOLUME bronze

In [0]:
# Função para normalizar nomes (eliminar caracteres não permitidos em nomes de colunas)
def normalize_columns(df):
    new_cols = [c.lower().replace(" ", "_") for c in df.columns]
    return df.toDF(*new_cols)

#Download do dataset dos aplicativos do Google Play
dbutils.fs.cp(
    "https://github.com/Romario76/Pos-PUCRJ-Data-Science-Analytics/raw/refs/heads/SPRINT_EngDADOS/googleplaystore.csv", 
    "dbfs:/Volumes/mvp/bronze/bronze/googleplaystore.csv")

#Conversão do arquivo googleplaystore.csv em um Dataframe do Spark
df_appBase = spark.read.option("header", True).option("inferSchema", True).csv("dbfs:/Volumes/mvp/bronze/bronze/googleplaystore.csv")

#Renomeação das colunas do Dataframe com caracteres inválidos
df_appBase_normalized = normalize_columns(df_appBase)

#Criação da tabela app_base no formato Delta na camada bronze
df_appBase_normalized.write.format("delta").mode("overwrite").saveAsTable("app_base")

print(f"Carga dos dados e criação da tabela bronze concluídos")


rows = df_appBase_normalized.count()           # total de linhas
cols = len(df_appBase_normalized.columns)      # total de colunas
print(f"O Dataset possui {rows} linhas e {cols} colunas.")


Carga dos dados e criação da tabela bronze concluídos
O Dataset possui 10841 linhas e 13 colunas.


### Catálogo de dados da tabela bronze

| Coluna         | Tipo de Dado | Descrição                                              | Domínio / Valores Esperados                |
|----------------|--------------|--------------------------------------------------------|--------------------------------------------|
| app            | string       | Nome do aplicativo definido pelo desenvolvedor         | Textual / Nome do aplicativo               |
| category       | string       | Nome da categoria a qual o aplicativo foi associado    | Textual / Nome da categoria                |
| rating         | string       | Avaliação geral dos usuários para cada aplicativo      | Numérico / 0-5                             |
| reviews        | string       | Total de avaliações do aplicativo                      | Numérico / valores                         |
| size           | string       | Tamanho do aplicativo                                  | Numérico / valores em kB ou MB             |
| installs       | string       | Total de downloads/instalações do aplicativo           | Numérico / valores                         |
| type           | string       | Indica se o aplicativo é gratuito ou pago              | Textual / Free ou Paid                     |
| price          | string       | Preço do aplicativo em dólares                         | Numérico / valores                         |
| content_rating | string       | Classificação indicativa do aplicativo                 | Textual / Faixas etárias                   |
| genres         | string       | Lista de gêneros associados ao aplicativo              | Textual / Nomes de gêneros                 |
| last_updated   | string       | Data da última atualização do app na Play Store        | Data / formato dd/mm/aaaa                  |
| current_ver    | string       | Versão atual do app disponível na Play Store           | Textual / Código identificador             |
| android_ver    | string       | Versão mínima requerida do Android                     | Textual / Código de uma versão válida      |


In [0]:
%sql
DESCRIBE TABLE mvp.bronze.app_base

col_name,data_type,comment
app,string,null
category,string,null
rating,string,null
reviews,string,null
size,string,null
installs,string,null
type,string,null
price,string,null
content_rating,string,null
genres,string,null


## 3. Camada Silver: Limpeza e Transformação
**Objetivo:** Transformar os dados brutos da Bronze em tabelas limpas, com tipos corrigidos, dados filtrados. A modelagem será mantida flat (uma tabela).

### Catálogo de dados

| Coluna         | Tipo de Dado | Descrição                                              | Domínio / Valores Esperados                |
|----------------|--------------|--------------------------------------------------------|--------------------------------------------|
| app            | string       | Nome do aplicativo definido pelo desenvolvedor         | Textual / Nome do aplicativo               |
| category       | string       | Nome da categoria a qual o aplicativo foi associado    | Textual / Nome da categoria                |
| rating         | double       | Avaliação geral dos usuários para cada aplicativo      | Numérico / 0-5                             |
| reviews        | bigint       | Total de avaliações do aplicativo                      | Numérico / valores                         |
| size           | double       | Tamanho do aplicativo                                  | Numérico / valores em kB ou MB             |
| installs       | bigint       | Total de downloads/instalações do aplicativo           | Numérico / valores                         |
| type           | string       | Indica se o aplicativo é gratuito ou pago              | Textual / Free ou Paid                     |
| price          | double       | Preço do aplicativo em dólares                         | Numérico / valores                         |
| content_rating | string       | Classificação indicativa do aplicativo                 | Textual / Faixas etárias                   |
| genres         | string       | Lista de gêneros associados ao aplicativo              | Textual / Nomes de gêneros                 |

In [0]:
%sql
USE CATALOG mvp;
USE SCHEMA silver

In [0]:
%sql
CREATE OR REPLACE TABLE silver.app_base_silver
USING delta
COMMENT 'Tabela contendo os dados do aplicativos Google Play da camada silver'
AS
SELECT
  app AS app_name,
  category,
  rating,
  reviews,
  size,
  price,
  content_rating,
  genres,
  regexp_replace(installs, '[+,]', '') AS installs, --limpeza de installs
  CASE -- Limpeza do status
    WHEN type = 'NaN' THEN null
    ELSE type
  END AS cost_type -- Renomeando 'type' para evitar conflito
  -- Colunas 'last_updated', 'current_ver' e 'android_ver' são colunas dimensionais que não impactam nas análises propostas e não serão utilizadas
FROM bronze.app_base

num_affected_rows,num_inserted_rows


### 3.1 Limpeza de nulos
Identificação e limpeza de valores nulos no dataset

In [0]:
%sql
SELECT 
    COUNT(CASE WHEN app_name IS NULL THEN 1 END) AS null_count_app_name,
    COUNT(CASE WHEN category IS NULL THEN 1 END) AS null_count_category,
    COUNT(CASE WHEN rating IS NULL THEN 1 END) AS null_count_rating,
    COUNT(CASE WHEN reviews IS NULL THEN 1 END) AS null_count_reviews,
    COUNT(CASE WHEN size IS NULL THEN 1 END) AS null_count_size,
    COUNT(CASE WHEN installs IS NULL THEN 1 END) AS null_count_installs,
    COUNT(CASE WHEN cost_type IS NULL THEN 1 END) AS null_count_cost_type,
    COUNT(CASE WHEN price IS NULL THEN 1 END) AS null_count_price,
    COUNT(CASE WHEN content_rating IS NULL THEN 1 END) AS null_count_content_rating,
    COUNT(CASE WHEN genres IS NULL THEN 1 END) AS null_count_genres
FROM silver.app_base_silver;

null_count_app_name,null_count_category,null_count_rating,null_count_reviews,null_count_size,null_count_installs,null_count_cost_type,null_count_price,null_count_content_rating,null_count_genres
0,0,0,0,0,0,1,0,1,0


**Resultado**

Localizado um registro em `content_rating` e um registro em `cost_type`, que podem pertencer a mesma linha ou não. Como são dois atributos de texto, não é possível determinar  os valores corretos. Pela baixa ocorrência, estes registros serão excluídos.

In [0]:
%sql
DELETE FROM silver.app_base_silver
WHERE content_rating IS NULL OR cost_type IS NULL

num_affected_rows
2


In [0]:
%sql
SELECT 
    COUNT(content_rating) AS null_count_content_rating,
    COUNT(cost_type) AS null_count_cost_type
FROM silver.app_base_silver
WHERE content_rating IS NULL OR cost_type IS NULL

null_count_content_rating,null_count_cost_type
0,0


### 3.2 Verificação de dados inválidos - campos numéricos
As colunas `rating`, `reviews`, `installs`, `price` e `size` possuem, como domínio natural, dados numéricos, porém o tipo de dados no _schema_ bronze é string.

Antes de realizar o ajuste do tipo de dados e as conversões dados, será avaliada a existência de valores inválidos, ou seja, fora do domínio dos números reais/naturais, e realizados os devidos tratamentos, para cada atributo.

In [0]:
%sql
SELECT --Teste de validação se não há dados inválidos (textos, símbolos etc)
    COUNT(CASE WHEN NOT regexp_like(rating, '^[+-]?[0-9]+(\\.[0-9]+)?$') THEN 1 END) AS invalid_count_rating,
    COUNT(CASE WHEN NOT regexp_like(reviews, '^[+-]?[0-9]+(\\.[0-9]+)?$') THEN 1 END) AS invalid_count_reviews,
    COUNT(CASE WHEN NOT regexp_like(installs, '^[+-]?[0-9]+(\\.[0-9]+)?$') THEN 1 END) AS invalid_count_installs,
    COUNT(CASE WHEN NOT regexp_like(price, '^[+-]?[0-9]+(\\.[0-9]+)?$') THEN 1 END) AS invalid_count_price,
    COUNT(CASE WHEN NOT regexp_like(size, '^[+-]?[0-9]+(\\.[0-9]+)?$') THEN 1 END) AS invalid_count_size
FROM silver.app_base_silver

invalid_count_rating,invalid_count_reviews,invalid_count_installs,invalid_count_price,invalid_count_size
1475,2,0,802,10839


### 3.2.1 Limpeza dos dados para cada coluna
Realizar a limpeza de dados inválidos garante integridade de domínio às colunas.

Neste pipeline, é realizada em duas etapas:
1) Análise dos dados gravados, visualizando de forma agrupada por valor, a distribuição dos erros.
2) Realização do tratamento mais adequado para cada erro (exclusão, transformação, substituição) e validação da execução da limpeza.

_a coluna `installs` não possui dados inválidos_

In [0]:
%sql
--Visualização dos dados inválidos, afim de definir a melhor estratégia de limpeza (exclusão ou substituição)
SELECT rating, COUNT(*)
FROM silver.app_base_silver
WHERE NOT regexp_like(rating, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY rating

rating,COUNT(*)
NaN,1473
navigation,1
Body,1


**Resultado**

Ocorrência de três grupos de dados inválidos.

Para as strings 'navigation' e 'Body', os registros serão excluídos da tabela.
Já os valores literais 'NaN' serão substituídos pela média.

In [0]:
%sql

--Apagar linhas com strings inválidas
DELETE FROM silver.app_base_silver
WHERE rating == ' navigation' OR rating == 'Body';

--SUBSTITUIÇÃO DOS VALORES NaN pela média
-- Cálculo da média de rating (excluindo 'NaN')
WITH avg_rating_cte AS (
  SELECT AVG(CAST(REGEXP_REPLACE(rating, '[^0-9.]', '') AS DOUBLE)) AS avg_rating
  FROM silver.app_base_silver
  WHERE rating != 'NaN'
)
-- Atualizando as linhas onde rating é 'NaN' com a média calculada
UPDATE silver.app_base_silver
SET rating = (SELECT avg_rating FROM avg_rating_cte)
WHERE rating == 'NaN';

--Teste de validação da limpeza
SELECT COUNT(rating)
FROM silver.app_base_silver
WHERE NOT regexp_like(rating, '^[+-]?[0-9]+(\\.[0-9]+)?$');    

COUNT(rating)
0


In [0]:
%sql
--Visualização dos dados inválidos, afim de definir a melhor estratégia de limpeza (exclusão ou substituição)
SELECT reviews, COUNT(*)
FROM silver.app_base_silver
WHERE NOT regexp_like(reviews, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY reviews

reviews,COUNT(*)


**Resultado**

No teste inicial, foram contabilizados dois registros inválidos. A execução da limpeza em `rating` indica que faziam parte do mesmo conjunto, pois não foi retornado valores.

In [0]:
%sql
--Visualização dos dados inválidos, afim de definir a melhor estratégia de limpeza (exclusão ou substituição)
SELECT price, count(price) AS invalid_count_price
FROM silver.app_base_silver
WHERE NOT regexp_like(price, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY price
ORDER BY count(*) DESC LIMIT 10

price,invalid_count_price
$0.99,148
$2.99,129
$1.99,73
$4.99,72
$3.99,63
$1.49,46
$5.99,30
$2.49,26
$9.99,21
$6.99,13


**Resultado**

Há dados carregados com o símbolo '$', o que impedirá a correta conversão numérica. 

In [0]:
%sql
--Eliminar literal monetário '$'
UPDATE silver.app_base_silver
SET price = replace(price, '$', '');

--Teste de validação da limpeza
SELECT count(price) AS invalid_count_price
FROM silver.app_base_silver
WHERE NOT regexp_like(price, '^[+-]?[0-9]+(\\.[0-9]+)?$')

invalid_count_price
0


In [0]:
%sql
--Visualização dos dados inválidos, afim de definir a melhor estratégia de limpeza (exclusão ou substituição)
SELECT size, COUNT(*) AS invalid_count_size
FROM silver.app_base_silver
WHERE NOT regexp_like(size, '^[+-]?[0-9]+(\\.[0-9]+)?$')
GROUP BY size
ORDER BY count(*) DESC
LIMIT 15;


size,invalid_count_size
Varies with device,1693
11M,198
12M,196
14M,194
13M,191
15M,184
17M,160
19M,154
16M,149
26M,149


**Resultado**

As linhas consideradas inválidas possuem as siglas 'k' (kilo) e 'M' (mega).

Estratégia: eliminar as siglas e colocar em uma mesma base (kilobytes).
Para os valores literais 'none', será realizada em duas etapas: a primeira altera o valor para null, a segunda substitui os nulos pela médias dos valores válidos de tamanho dos aplicativos do dataset. Por segurança, serão incluídas siglas 'G' (giga) e 'T' (tera).

In [0]:
%sql
-- Converte strings como '10M', '512K', '1G' para kB(kilobytes) (base 1024)
-- try_cast retorna null para valores inválidos ('none')
-- regexp_replace remove todos os caracteres não numéricos
UPDATE silver.app_base_silver
SET size = try_cast(regexp_replace(size, '[^0-9.]', '') AS DOUBLE) *
  CASE
    WHEN regexp_like(size, '.*[Mm]$') THEN POW(1024, 1)
    WHEN regexp_like(size, '.*[Gg]$') THEN POW(1024, 2)
    WHEN regexp_like(size, '.*[Tt]$') THEN POW(1024, 3)
    ELSE 1
  END
;

-- Substituição dos valores NULL pela média
-- Cálculo da média de size (excluindo 'null')
WITH avg_size_cte AS (
  SELECT AVG(size) AS avg_size
  FROM silver.app_base_silver
  WHERE size IS NOT NULL
)

-- Atualizando as linhas NULL com a média calculada
UPDATE silver.app_base_silver
SET size = (SELECT avg_size FROM avg_size_cte)
WHERE size IS NULL;

--Teste de validação da limpeza
SELECT
    COUNT(size) AS invalid_count_size
FROM silver.app_base_silver
WHERE NOT regexp_like(size, '^[+-]?[0-9]+(\\.[0-9]+)?$');

invalid_count_size
0


### 3.3 Ajuste do schema da tabela silver
Com os dados limpos e convertidos, será realizado o ajuste do tipo de dados no schema para INT ou DOUBLE ou equivalente.

Através do PySpark, a tabela foi carregada em um dataframe, alterando os tipos de dados dos atributos necessários e, por fim, sobreescrevendo o schema da tabela na camada silver.

Usando SQL, as alternativas seriam, (1) criar uma nova tabela com o schema final, replicar os dados nesta tabela, descartar a original e renomear a nova, ou (2) realizar uma operação semelhante, porém na própria tabela, criando novas colunas com os tipos de dados finais, copiando os dados, excluindo as colunas originais e renomeando cada coluna criada.

In [0]:
#Iniciando uma váriavel com uma string do nome da tabela para reaproveitamento
table = "silver.app_base_silver"

In [0]:
#carregar a tabela silver em um dataframe
df_Str = spark.table(table)

#alterar os tipos de dados das colunas numéricas
df_Num = df_Str.withColumn("rating",F.col("rating").cast("double")) \
    .withColumn("reviews",F.col("reviews").cast("long")) \
    .withColumn("size",F.col("size").cast("double")) \
    .withColumn("installs",F.col("installs").cast("long")) \
    .withColumn("price",F.col("price").cast("double"))

display(df_Num.dtypes)

_1,_2
app_name,string
category,string
rating,double
reviews,bigint
size,double
price,double
content_rating,string
genres,string
installs,bigint
cost_type,string


### 3.4 Tratamento de duplicidades
O dataset não possui uma coluna identificadora, p.e., 'id' ou 'código app'. 

Na extração dos dados, pode ter ocorrido duplicação de registros.
Para garantir maior integridade e uma melhor veracidade, as duplicidades devem ser tratadas, ou seja removidas.


**3.4.1 Remover registros duplicados (todos atributos iguais)**

In [0]:
# Remove duplicados em todas as colunas
df_dedup = df_Num.dropDuplicates()  # equivalente a DISTINCT em todas as colunas

# Auditoria: confirmar a remoção de registros
antes = df_Num.count()
depois = df_dedup.count()
print(f"Linhas antes: {antes} | Linhas depois: {depois} | Removidas: {antes - depois}")

Linhas antes: 10837 | Linhas depois: 10352 | Removidas: 485


In [0]:
#Salva as linhas removidas antes de sobrescrever:
cols = df_dedup.columns  # todas as colunas
df_removed = df_Num.exceptAll(df_dedup)
df_removed.write.format("delta").mode("overwrite").saveAsTable("silver.app_base_removidas")

# Confirmar que os registros duplicados foram exluídos

df_sig = df_dedup.withColumn("assinatura", F.md5(F.concat_ws("||", *[F.col(c).cast("string") for c in cols]))) #cria string única para cada linha

df_dup = (df_sig
          .groupBy("assinatura")
          .agg(F.count(F.lit(1)).alias("qtd"))
          .filter(F.col("qtd") > 1)
          .orderBy(F.desc("qtd"))) #conta número de linhas duplicadas

df_dup.show(50, truncate=False) # Resultado = 0 ou vazio, confirma sucesso da execução


+----------+---+
|assinatura|qtd|
+----------+---+
+----------+---+



**3.4.2 Identificação de outras duplicidades**

Os nomes de aplicativos podem se repetir,p.e., 'Solitaire', salvo se estiver protegido por propriedade intelectual (Instagram, Whatsapp).
Porém, há outros atributos que, se agregado, pode tornar o registro único.

Para cada aplicativo com mais de uma linha, avalia-se a existência de repetições dos demais atributos da tabela

Para minimizar perda de dados no tratamento de duplicidades "parciais", adotou-se a linha mais recente como a mais confiável. Em tese, dados mais novos terão maiores quantidades de avaliações e instalações, e, possivelmente, maior tamanho. Então, os atributos `reviews`, `installs` e `size` serão ordenados em ordem descrescente. 

Salvaguardas para evitar mesclas indevidas: a chave será modificada com a inclusão do campo `cost_type`.

Justificativa: Mesmo que seja o mesmo aplicativo, com versões gratuita e paga,uma versão paga pode ter dados diferentes em relação à gratuita (`rating`, `reviews`, `installs`). Esta informação pode ser relevante na tomada de decisão de cobrar ou não pela instalação de um aplicativo.

In [0]:
# Visualizar outras duplicidades pela chave (nome do app)
key = "app_name"
vc = [c for c in df_dedup.columns if c != key]

# Estatísticas por coluna (linhas, distintos e duplicados)
# Conta linhas por chave
rpkey = df_dedup.groupBy(key).agg(F.count(F.lit(1)).alias("linhas_no_grupo"))

# Seleciona as linhas com mais de uma linha por chave
rpkey_multi = [] #criar lista para armazenar as chaves com mais de uma linha
for row in rpkey.collect():
    k = row[key]
    l = row["linhas_no_grupo"]
    if l > 1:
        rpkey_multi.append((k,l))

#converte a lista em dataframe
df_multi_key = spark.createDataFrame(rpkey_multi, [key,"linhas_no_grupo"])

df_display = df_multi_key.orderBy(F.col("linhas_no_grupo").desc()).limit(10)
display(df_display)


app_name,linhas_no_grupo
ROBLOX,9
8 Ball Pool,7
Helix Jump,6
Zombie Catchers,6
Bubble Shooter,6
Temple Run 2,5
Candy Crush Saga,5
Subway Surfers,5
Angry Birds Classic,5
Granny,5


In [0]:

# Relatório por coluna, agrupado por chave duplicada

key = "app_name" # define a chave
value_cols = [c for c in df_dedup.columns if c != key]

# Conta distintos por coluna (uma agregação única com várias expressões)
distinct_exprs = [
    F.countDistinct(F.col(c)).alias(f"distinct__{c}")
    for c in value_cols
]
distinct_per_key = df_dedup.groupBy(key).agg(*distinct_exprs)

# Junta e calcula duplicados por coluna
stats = df_multi_key.join(distinct_per_key, on=key, how="inner")

# Cria um array com os nomes das colunas que têm duplicidade por chave
dup_flags = [
    F.when(F.col("linhas_no_grupo") - F.col(f"distinct__{c}") > 0, F.lit(c))
    for c in value_cols
]
stats = stats.withColumn("colunas_com_duplicidade", F.array(*dup_flags))
stats = stats.withColumn(
    "colunas_com_duplicidade",
    F.expr("filter(colunas_com_duplicidade, x -> x is not null)")
)

# Cria um array com colunas sem duplicidade (constantes ou totalmente distintas)
no_dup_flags = [
    F.when(F.col("linhas_no_grupo") - F.col(f"distinct__{c}") == 0, F.lit(c))
    for c in value_cols
]
stats = stats.withColumn("colunas_sem_duplicidade", F.array(*no_dup_flags))
stats = stats.withColumn(
    "colunas_sem_duplicidade",
    F.expr("filter(colunas_sem_duplicidade, x -> x is not null)")
)

# Mostra resultado
select_cols = [key, "linhas_no_grupo", "colunas_com_duplicidade", \
    "colunas_sem_duplicidade"] + [f"distinct__{c}" for c in value_cols]

df_stats = stats.select(*select_cols).orderBy(F.col("linhas_no_grupo").desc()).limit(10)
display(df_stats)

app_name,linhas_no_grupo,colunas_com_duplicidade,colunas_sem_duplicidade,distinct__category,distinct__rating,distinct__reviews,distinct__size,distinct__price,distinct__content_rating,distinct__genres,distinct__installs,distinct__cost_type
ROBLOX,9,"List(category, rating, reviews, size, price, content_rating, genres, installs, cost_type)",List(),2,1,8,1,1,1,1,1,1
8 Ball Pool,7,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),2,1,7,1,1,1,1,1,1
Helix Jump,6,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,6,1,1,1,1,1,1
Bubble Shooter,6,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),2,2,6,3,1,1,3,2,1
Zombie Catchers,6,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,6,1,1,1,1,1,1
Angry Birds Classic,5,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,5,1,1,1,1,1,1
Subway Surfers,5,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,5,1,1,1,1,1,1
Temple Run 2,5,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,5,1,1,1,1,1,1
slither.io,5,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,5,1,1,1,1,1,1
Granny,5,"List(category, rating, size, price, content_rating, genres, installs, cost_type)",List(reviews),1,1,5,1,1,1,1,1,1


In [0]:
# Ajusta a chave, incluindo "cost_type"
group_keys = ["app_name","cost_type"]  

# Definição da LINHA BASE por chave (= mais recente)
w = Window.partitionBy(*group_keys).orderBy(
    F.col("reviews").desc_nulls_last(),
    F.col("installs").desc_nulls_last(),
    F.col("size").desc_nulls_last()
)
df_base = df_dedup.withColumn("rn", F.row_number().over(w)).filter(F.col("rn") == 1).drop("rn")

# Agregações por chave para não perder dados
agg = (df_dedup.groupBy(*group_keys)
         .agg(
             F.max("reviews").alias("reviews_max"),
             F.max("installs").alias("installs_max"),
             F.max("size").alias("size_max"),
             F.count("*").alias("rows_in_group"),
             F.countDistinct("category").alias("category_distinct"),
             F.collect_set("category").alias("category_set"),
             F.collect_set("genres").alias("genres_set"),
             F.collect_set("size").alias("size_set")
         ))

# Construir o REGISTRO CANÔNICO mesclando métricas no df_base. Este será o resultado que será persistido na tabela silver
df_canon = (df_base
            .join(agg, on=group_keys, how="inner")
            .withColumn("reviews", F.col("reviews_max"))
            .withColumn("installs", F.col("installs_max"))  
            .withColumn("size", F.col("size_max"))
            # Rating: mantém o da base
           )


In [0]:
# Remove colunas auxiliares, mantendo apenas as que serão persistidas na tabela (originais) 
df_canon = df_canon.drop(
  "reviews_max",
  "installs_max",
  "size_max",
  "rows_in_group",
  "category_distinct",
  "category_set",
  "genres_set",
  "size_set"
)


In [0]:

cols = df_dedup.columns
df_orig_h = df_dedup.withColumn("row_hash", sha2(concat_ws("§", *[col(c) for c in cols]), 256))
df_dedup_h = df_canon.withColumn("row_hash", sha2(concat_ws("§", *[col(c) for c in cols]), 256))

# Multiconjunto removido (diferença nas contagens por hash)
orig_counts = df_orig_h.groupBy("row_hash").agg(count("*").alias("n_orig"))
dedup_counts = df_dedup_h.groupBy("row_hash").agg(count("*").alias("n_dedup"))

rem_hashes = orig_counts.join(dedup_counts, "row_hash", "left_outer") \
                        .fillna({'n_dedup': 0}) \
                        .filter(col("n_orig") > col("n_dedup")) \
                        .select("row_hash", "n_orig", "n_dedup")

# Recuperando as linhas removidas (limitando pela diferença)
removidas = df_orig_h.join(rem_hashes.select("row_hash"), "row_hash", "inner").drop("row_hash")


In [0]:
df_audit = (agg
            .select(*group_keys, "rows_in_group", "category_set", "genres_set", "size_set")
            .withColumn("observacao",
                        F.when(F.col("rows_in_group") > 1, F.lit("havia duplicidade e foi consolidado"))
                         .otherwise(F.lit("sem duplicidade")))
           )
# Auditoria: salvar tabelas auxiliares Delta
df_canon.write.format("delta").mode("overwrite").saveAsTable("app_canon")
removidas.write.format("delta").mode("overwrite").saveAsTable("app_dedup_removidas")
df_audit.write.format("delta").mode("overwrite").saveAsTable("app_auditoria")

# Auditoria: confirmar a remoção de registros
antes = df_dedup.count()
depois = df_canon.count()
print(f"Linhas antes: {antes} | Linhas depois: {depois} | Removidas: {antes - depois}")

# Relatório rápido
df_show = df_audit.orderBy(F.col("rows_in_group").desc()).limit(10)
display(df_show)


Linhas antes: 10352 | Linhas depois: 9658 | Removidas: 694


app_name,cost_type,rows_in_group,category_set,genres_set,size_set,observacao
ROBLOX,Free,9,"List(GAME, FAMILY)",List(Adventure;Action & Adventure),List(68608.0),havia duplicidade e foi consolidado
8 Ball Pool,Free,7,"List(GAME, SPORTS)",List(Sports),List(53248.0),havia duplicidade e foi consolidado
Helix Jump,Free,6,List(GAME),List(Action),List(33792.0),havia duplicidade e foi consolidado
Zombie Catchers,Free,6,List(GAME),List(Action),List(76800.0),havia duplicidade e foi consolidado
Bubble Shooter,Free,6,"List(GAME, FAMILY)","List(Casual, Puzzle;Brain Games, Arcade)","List(47104.0, 20480.0, 51200.0)",havia duplicidade e foi consolidado
Temple Run 2,Free,5,List(GAME),List(Action),List(63488.0),havia duplicidade e foi consolidado
Candy Crush Saga,Free,5,"List(GAME, FAMILY)",List(Casual),List(75776.0),havia duplicidade e foi consolidado
Subway Surfers,Free,5,List(GAME),List(Arcade),List(77824.0),havia duplicidade e foi consolidado
Angry Birds Classic,Free,5,List(GAME),List(Arcade),List(99328.0),havia duplicidade e foi consolidado
Granny,Free,5,List(GAME),List(Arcade),List(60416.0),havia duplicidade e foi consolidado


### 3.5 Atualização da tabela silver com os dados limpos

In [0]:
# Grava os dados limpos na tabela silver
(df_canon
  .write
  .format("delta")
  .mode("overwrite")      # sobrescreve o conteúdo
  .option("overwriteSchema", "true")
  .saveAsTable(table))


### 3.6 Adicionando comentários na tabela (metadados)

In [0]:
appBase_silver_comments = {
    "app_name": "Nome do aplicativo definido pelo desenvolvedor",
    "category": "Nome da categoria a qual o aplicativo foi associado",
    "rating": "Avaliação geral dos usuários para cada aplicativo",
    "reviews": "Total de avaliações do aplicativo",
    "size": "Tamanho do aplicativo em kB",
    "price": "Preço do aplicativo em dólares",
    "content_rating": "Classificação indicativa do aplicativo",
    "genres": "Lista de gêneros associados ao aplicativo",
    "installs": "Total de downloads/instalações do aplicativo",
    "cost_type": "Indica se o aplicativo é gratuito ou pago"
}

for col_name, comment in appBase_silver_comments.items():
    try:
        spark.sql(f"ALTER TABLE {table} ALTER COLUMN {col_name} COMMENT '{comment}'")
    except Exception as e:
        print(f"  Aviso: Não foi possível aplicar comentário para a coluna '{col_name}'. Erro: {e}")

print("\n✅ Comentários aplicados às colunas das tabelas Silver.")


✅ Comentários aplicados às colunas das tabelas Silver.


In [0]:
%sql
DESCRIBE TABLE silver.app_base_silver

col_name,data_type,comment
app_name,string,Nome do aplicativo definido pelo desenvolvedor
cost_type,string,Indica se o aplicativo é gratuito ou pago
category,string,Nome da categoria a qual o aplicativo foi associado
rating,double,Avaliação geral dos usuários para cada aplicativo
reviews,bigint,Total de avaliações do aplicativo
size,double,Tamanho do aplicativo em kB
price,double,Preço do aplicativo em dólares
content_rating,string,Classificação indicativa do aplicativo
genres,string,Lista de gêneros associados ao aplicativo
installs,bigint,Total de downloads/instalações do aplicativo


## 4 Camada gold: Agregação e Análise de Negócios
Nesta camada, são criadas tabelas agregadas ou visualizações específicas para responder às perguntas de negócio levantadas. As tabelas Gold são a fonte para dashboards e relatórios.

**Tabela Gold: category_analysis**

Catálogo de dados
| Coluna            | Tipo de Dado | Descrição                                         | Domínio / Valores Esperados      |
|-------------------|--------------|---------------------------------------------------|----------------------------------|
| category          | string       | Nome da categoria a qual o aplicativo foi associado| Textual / Nome da categoria      |
| num_apps          | bigint       | Total de aplicativos                              | Numérico / valores               |
| avg_rating        | double       | Média das avaliações de cada aplicativo           | Numérico / 0-5                   |
| total_reviews     | bigint       | Soma das avaliações                               | Numérico / valores               |
| total_installs    | bigint       | Soma dos downloads                                | Numérico / valores               |
| percent_per_cat   | double       | Proporção por categoria                           | Numérico / valores               |

In [0]:
%sql
CREATE OR REPLACE TABLE gold.category_analysis
USING delta
COMMENT 'Análise para entender como as categorias estão distribuídas e como elas se comportam em termos de avaliações, classificações e instalações.'
AS
SELECT
  category,
  COUNT(*) AS num_apps,
  AVG(rating) AS avg_rating,
  SUM(reviews) AS total_reviews,
  SUM(installs) AS total_installs,
  num_apps / (SELECT COUNT(*) FROM silver.app_base_silver) AS percent_per_cat
FROM silver.app_base_silver
GROUP BY category


num_affected_rows,num_inserted_rows


### 4.1 Análise: Distribuição de aplicativos por categoria
Quais categorias possuem maiores quantidades de aplicativos disponíveis no catálogo da Google Play?

In [0]:
%sql
SELECT 
  category AS Categoria, 
  num_apps AS Qtd_app,
  ROUND(percent_per_cat*100,1) AS Percentual
FROM gold.category_analysis 
ORDER BY num_apps DESC LIMIT 10;



Categoria,Qtd_app,Percentual
FAMILY,1875,19.4
GAME,946,9.8
TOOLS,829,8.6
BUSINESS,420,4.3
MEDICAL,396,4.1
PERSONALIZATION,376,3.9
PRODUCTIVITY,374,3.9
LIFESTYLE,369,3.8
FINANCE,345,3.6
SPORTS,325,3.4


**Insights**

Muitos aplicativos foram categorizados como "Familiar", ou seja voltados para famílias, seguido pela "Jogos".

Se por um lado, escolher criar um aplicativo dentro das primeiras categorias possam dar maior visibilidade por, talvez, serem as mais procuradas, por outro lado, novos aplicativos precisam possuir diferenciais para conseguirem se destacar, já que a concorrência é bem maior.

### 4.2 Análise: Classificação dos usuário
Alguma categoria se destaca em termos de classificação?

In [0]:
%sql
SELECT category AS Categoria, ROUND(avg_rating,3) AS Media_Avaliacoes
FROM gold.category_analysis 
ORDER BY avg_rating DESC LIMIT 10;

Categoria,Media_Avaliacoes
EVENTS,4.363
ART_AND_DESIGN,4.354
EDUCATION,4.348
BOOKS_AND_REFERENCE,4.308
PERSONALIZATION,4.303
PARENTING,4.282
BEAUTY,4.261
GAME,4.242
SOCIAL,4.239
WEATHER,4.239


**Insights**

Nenhuma categoria se destaca muito, a diferença entre as dez primeira 0,13 pontos. Nota-se também não há relação entre quantidade e qualidade dos aplicativos, pois apenas duas categorias com maior número de aplicativos estão entre as dez melhores avaliadas.

Para tomada de decisão, esta métrica não contribui com informações relevantes.

### 4.3 Análise: Quantidade de avaliações na Play Store
E em relação ao número de avaliações, quais possuem maior engajamento dos usuários?

In [0]:
%sql
SELECT category AS Categoria, total_reviews AS Total_Avaliacoes
FROM gold.category_analysis 
ORDER BY total_reviews DESC LIMIT 10;

Categoria,Total_Avaliacoes
GAME,590772190
COMMUNICATION,285824082
TOOLS,229471630
SOCIAL,227936355
FAMILY,193543707
PHOTOGRAPHY,105580472
VIDEO_PLAYERS,67644444
PRODUCTIVITY,55590856
PERSONALIZATION,53543117
SHOPPING,44554067


**Insights**

O resultado mostra que usuários de aplicativos de "Jogos" estão mais engajados em mostrar suas opiniões, mesmo que possam ser negativas.
Isso pode ajudar o desenvolvedor a trazer melhorias para seu aplicativo. 

Conectar esta métrica com os comentários dos usuários, poderá trazer informações mais relevantes, ao estruturar as opiniões por palavras-chave, tanto para elogios, quanto para queixas e sugestões.

### 4.4 Análise: popularidade
Qual a distribuição de downloads por categoria?

In [0]:
%sql
SELECT category AS Categoria, total_installs AS Total_Downloads
FROM gold.category_analysis 
ORDER BY total_installs DESC LIMIT 10;

Categoria,Total_Downloads
GAME,13457924415
COMMUNICATION,11038276251
TOOLS,8102771915
FAMILY,6222542505
PRODUCTIVITY,5793091369
SOCIAL,5487867902
PHOTOGRAPHY,4658147655
VIDEO_PLAYERS,3931902720
TRAVEL_AND_LOCAL,2894887146
NEWS_AND_MAGAZINES,2369217760


**Insight**

Mais uma métrica que indica que quantidade pode não signifcar qualidade.
Apesar da categoria "Familiar" possuir quase o dobro de "Jogos", observando o total de instalações, "Jogos" que passa a ter quase o dobro de "Familiar", este ainda sendo ultrapassado por outras duas categorias ("Comunicação" nem estava entre os Top10 em quantidade).

Assim como na métrica de quantidade e aplicativos, deve-se avaliar questões de visibilidade x concorrência. Categoria populares constumam tem grandes desenvolvedores e ter um diferencial poderá ser crucial para o sucesso, ou não, de um novo aplicativo.

**Tabela Gold: cost_type_analysis**

Catálogo de dados

| Coluna            | Tipo de Dado | Descrição                                   | Domínio / Valores Esperados      |
|-------------------|--------------|---------------------------------------------|----------------------------------|
| cost_type         | string       | Indica se o aplicativo é gratuito ou pago   | Textual / Free ou Paid           |
| num_apps          | bigint       | Total de aplicativos                        | Numérico / valores               |
| total_reviews     | bigint       | Soma das avaliações                         | Numérico / valores               |
| total_installs    | bigint       | Soma dos downloads                          | Numérico / valores         |
| percent_per_type  | double       | Proporção por tipo de aplicativo            | Numérico / valores               |

In [0]:
%sql
CREATE OR REPLACE TABLE gold.cost_type_analysis
USING delta
COMMENT 'Análise para avaliar distribuição entre apps pagos e gratuitos e se há influência na quantidade de avaliações e instalações'
AS
SELECT
  cost_type,
  COUNT(app_name) AS num_apps,
  SUM(reviews) AS total_reviews,
  SUM(installs) AS total_installs,
  num_apps / (SELECT COUNT(*) FROM silver.app_base_silver) AS percent_per_type
FROM silver.app_base_silver
GROUP BY cost_type

num_affected_rows,num_inserted_rows


### 4.5 Análise: pago ou gratuito?
Alguns aplicativos são pagos e outros gratuitos. Quais são os mais baixados em cada tipo?

In [0]:
%sql
SELECT 
  cost_type AS Tipo, 
  num_apps AS Qtd_app,
    ROUND(percent_per_type*100,1) as Percentual,
  total_installs As Downloads
FROM gold.cost_type_analysis 
GROUP BY cost_type, num_apps, percent_per_type, total_installs
ORDER BY num_apps DESC;

Tipo,Qtd_app,Percentual,Downloads
Free,8902,92.2,75259162646
Paid,756,7.8,57364881


**Insights**

Como esperado, a quantidade de aplicativos gratuitos disponíveis na plataforma é muito maior, refletindo na quantidade instalada.

Salvo em caso de uma aplicativo de nincho, voltado para algum público específico, o retorno finaceiro deve ser avaliado em cima de cotas de propagandas (advertising) ou compras dentro do aplicativo.

E o engajamento, qual tipo possui maior quantidade de avaliações em relação à quantidade downloads?

In [0]:
%sql
SELECT
  cost_type AS Tipo,
  ROUND(total_reviews / total_installs*100,1) AS perc_aval_down
FROM gold.cost_type_analysis
GROUP BY cost_type, total_reviews, total_installs
ORDER BY perc_aval_down DESC;

Tipo,perc_aval_down
Paid,11.5
Free,2.8


**Insights**

Proporcionalmente, a quantidade de usuários que realizaram avaliações na Play Stores para aplicativos pagos é maior. Demonstra que há preocupação em dar um feedback por ter havido desembolso financeiro para obter o aplicativo.

Conectar esta métrica com os comentários dos usuários, poderá trazer informações mais relevantes, ao estruturar as opiniões por palavras-chave, tanto para elogios, quanto para queixas e sugestões, pois pode haver uma tendência de maior exigência de qualidade.

## 5 Autoavaliação e conclusão


Este MVP foi o mais desafiador para mim nesta jornada da Pós de Ciência de Dados e Analytics da PUC-Rio. Além de novos conceitos, foi preciso aprender a usar a plataforma Databricks com o “carro andando”.
Meu objetivo com este curso é realizar uma migração de carreira para área de dados. Apesar de ter formação na área de TI, estou fora deste mercado há muitos anos. A área de dados está sendo um caminho que visualizei para essa mudança. Tenho contato com algum volume de dados, mas com atuação muito centrada em Excel. 
Um dos grandes desafios foi obter o conhecimento necessário para além do material da sprint, que fornece uma boa base, mas, na prática, precisamos buscar outras fontes. 
Outro conceito que tive que assimilar rapidamente foi o de pipeline de dados. Nestas horas, ter pouco conhecimento prévio pode até ajudar, pois o entendimento da arquitetura Medalhão (Bronze, Silver, Gold) foi o menos problemático.
No meu trabalho, busquei alternar o uso de SQL e PySpark para assimilar os conceitos de Spark (SQL e PySpark – e descobrir que o dataframe Spark é diferente do Pandas), Delta Lake, modelagem e as melhores práticas de engenharia de dados.
Por fim, esta sprint foi um grande desafio pessoal e sinto uma grande satisfação por ter concluído este projeto apesar de todas as dificuldades.
---

### Objetivos Alcançados:

*   **Pipeline Completo:** Construí com sucesso o pipeline de dados seguindo a arquitetura Medalhão, com schemas `bronze`, `silver` e `gold` bem definidos e tabelas Delta Lake em cada camada.
*   **Respostas de Negócio:** As perguntas de negócio propostas inicialmente foram respondidas através de análises materializadas em tabelas na camada Gold.
*   **Documentação:** O processo foi documentado no próprio notebook através das células Markdown e comentários no código, explicando a lógica e o propósito de cada etapa.

### Principais Dificuldades Superadas:

*   **Inconsistências nos Dados:** Lidar com dados brutos que apresentam inconsistências, tratamento de nulos e duplicidades.
*   **Curva de Aprendizado:** Absorver e aplicar efetivamente os conceitos e ferramentas da engenharia de dados moderna (Spark, Delta Lake, SQL).
---

### Conclusão
Com muita persistência e resiliência, consegui finalizar este MVP. Acredito ter aplicado as boas práticas aprendidas nas outras sprints, tanto na codificação quanto na documentação. Infelizmente, não consegui trazer os outputs visuais, que traria um aspecto mais profissional e gerencial. O dataset utilizando talvez permita a modelagem estrela. Então, são dois pontos que poderei incrementar neste trabalho.
Mas como aprendi por onde passei, o ótimo é inimigo do bom e antes ter um entregável do que não entregar nada. Aliás, é justamente esse o conceito do MVP, ter um entregável mínimo e trabalhar para torná-lo cada vez melhor e mais funcional.

